In [9]:
file_name="BR236-A236_1SR_DEF_6W-PP_SAE.inc"

In [10]:
import open3d as o3d
import numpy as np
import subprocess

In [ ]:
subprocess.run(["g++", "readF.cpp", "-o", "./a.exe"])
cpp_args = [str(),".ply"]
output=subprocess.run(["./a.exe"]+cpp_args, stdout=subprocess.PIPE, text=True)

In [3]:
def furthest_point_sampling(pcd,num_points):
    '''
    Furthest Point Sampling Algorithm
    Takes as input:
    pcd->point cloud(.ply) file
    num_points->user defined number of points for down-sampling
    Returns:new down-sampled point cloud(.ply)
    '''
    distances = pcd.compute_nearest_neighbor_distance()
    farthest_indices = np.zeros((num_points,), dtype=np.int32)
    seed = 0
    farthest_indices[0] = seed
    for i in range(1, num_points):
        seed = int(np.argmax(distances))
        farthest_indices[i] = seed
        distances[seed] = -1 
    downpcd=pcd.select_by_index(farthest_indices)
    cl, index = downpcd.remove_statistical_outlier(nb_neighbors=400,std_ratio=2)
    #outlier_cloud = downpcd.select_by_index(index, invert=True)
    return downpcd.select_by_index(index)
    

In [4]:
def outlier_remove(pcd,nb_neighbours,std_ratio):
    cl, index = pcd.remove_statistical_outlier(nb_neighbours,std_ratio)
    inlier_cloud = pcd.select_by_index(index)
    outlier_cloud = pcd.select_by_index(index, invert=True)
    return pcd.select_by_index(index)

In [2]:
pcd=o3d.io.read_point_cloud("p1.ply")
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PLY failed: unable to read file: p1.ply


In [8]:
file_name="p1.ply"
num_points=20000
pcd=o3d.io.read_point_cloud(file_name)
dpcd=furthest_point_sampling(pcd,num_points)
dpcd=outlier_remove(dpcd,5,5)
print([dpcd])
o3d.visualization.draw_geometries([dpcd])

[Open3D WARNING] Read PLY failed: unable to read file: p1.ply
[PointCloud with 19430 points.]
